In [4]:
from sage import *

In [6]:
N = 25898966400928827905718377946331123070958718286581765316565582158865227877882475404853218079499084099440419144196215764927720893687968939899067275095801562867742359933997487928281899714724738097735994026225339488710478292473051567851786254924548138570069406420407124627567648479424564834446192417334669768477661434992797176428220265984651288944265998446714590797833756720922745187467388408600309665467669255896919554072379878017822219455974525233467767926938557154083882126002952139561283708342676308894318951822068027821029295524097544028901807902120777407151278396388621981625398417573347316888458337381776303199529;

e = 1048577;

ciphertext_Blex = 11140520553087800834883326476247582685177207584737264356946559762068509060522907835540767944557089926814767920501376431871780404000550271362410228709616559148950928004959648199391157781102695421411667843970881959939688515679415870087007797819271601359811630724878746762862603629420061133824605384527474682526549557804674160851967543475275374840169790764048711047622418045734436512050742433282306694490346907876574514077395835974083376649624559301087384766644865104383786285302561584731767419571603248493060257358632833957327996996960955767927114473513709882904104552609194519132931270741118197821776138632855021619178;

ciphertext_Kane = 2922817623733019475805146570530296261205732600738503605503192845278422660686627490817081424885152809772315629265930072636690234953045955503581182067349322827011065359648958225896393305011175960879100475146203207801533980643261035226402857047007061320653920746872424363923515091038846823007819033456503365649022294092944985887626605207259444051959239244136999684366533551627508385114998024232490369665950339127904350803268889205463047713233591604324960184727360413931125906144631968128488876241314939855024305076160092193380013725939761970042406866169417457376487954247442308318888399299295082898238584625937490546472;

R = PolynomialRing(Zmod(N), names='x')
x = R.gen()

mb = Bin(b"Blex").int
mk = Bin(b"Kane").int

e = 123
e = 1123 # 2s
e = 10123 # 20 s
e = 100123 # 280 s
e = 1048577 # ? 2000 s?

AttributeError: module 'sage' has no attribute 'PolynomialRing'

In [41]:
pb = (x + mb)**e - ciphertext_Blex
pk = (x + mk)**e - ciphertext_Kane

#a(x) * x**t + b(x) 
#c(x) * x**t + d(x)

In [35]:
def xgcd(a, b):
    assert a and b
    px, ppx = 0, 1
    py, ppy = 1, 0   
    while b:
        r = a % b
        q = (a - r) // b
        a, b = b, r
        x = ppx - q * px
        y = ppy - q * py
        ppx, px = px, x
        ppy, py = py, y
    return px, py, b, ppx, ppy, a

def semi(a, b):
    #print("semi", a.degree(), b.degree())
    assert a or b
    assert abs(a.degree() - b.degree()) <= 1
    a0, b0 = a, b
    px, ppx = 0, 1
    py, ppy = 1, 0
    if not a or not b:
        return px, py, b, ppx, ppy, a
    
    lim = (a.degree() + 1) // 2
    while b.degree() >= lim:
        #r = a % b
        #q = (a - r) // b
        q = a // b
        r = a - q * b
        a, b = b, r
        x = ppx - q * px
        y = ppy - q * py
        ppx, px = px, x
        ppy, py = py, y
    assert px * a0 + py * b0 == b
    assert ppx * a0 + ppy * b0 == a
    return px, py, b, ppx, ppy, a

def semi_large(a, b):
    assert abs(a.degree() - b.degree()) <= 1
    a0, b0 = a, b
    if a.degree() < 30:
        return semi(a, b)
    
    #print("semi large", a.degree(), b.degree())
    # px * a + py * b = b
    # ppx * a + ppy * b = a
    ans_px = 0
    ans_py = 1
    ans_ppx = 1
    ans_ppy = 0
    
    for i in range(2):
        #sage: (1-9/30)**2
        #49/100

        h = a.degree() * (30 - 2*9) // 30 
    
        la = list(a)
        lb = list(b)
        topa = R(la[h:])
        topb = R(lb[h:])
        
        #print("i", i, "semi large", a0.degree(), b0.degree(), "call h", h, "cur", a.degree(), b.degree(), "call")
        px, py, _b, ppx, ppy, _a = semi_large(topa, topb)
        b, a = px * a + py * b, ppx * a + ppy * b
        #print("i", i, "semi large", a0.degree(), b0.degree(), "call h", h, "ret", a.degree(), b.degree())
        
        '''
        ppx * old a + ppy * old b = new a
        px * old a + py * old b = new b
        ans_ppx * orig a + ans_ppy * orig b = old a
        ans_px * orig a + ans_py * orig b = old b
        '''
        
        ans_px, ans_py, ans_ppx, ans_ppy = (
            px * ans_ppx + py * ans_px,
            px * ans_ppy + py * ans_py,
            ppx * ans_ppx + ppy * ans_px,
            ppx * ans_ppy + ppy * ans_py,
        )
    return ans_px, ans_py, b, ans_ppx, ans_ppy, a

In [36]:
px, py, b, ppx, ppy, a = semi_large(pb, pk)
#assert px * pb + py * pk == b
#assert ppx * pb + ppy * pk == a

In [42]:
a, b = pb, pk
W = 2**16
while a.degree() > 2*W:
    print(a.degree(), b.degree())
    la = list(a)
    lb = list(b)
    d = a.degree()
    
    px, py, _, ppx, ppy, _ = semi_large(R(la[d-W:]), R(lb[d-W:]))
    print("muling...")
    b, a = px * a + py * b, ppx * a + ppy * b
    print("->", a.degree(), b.degree())
    print()
    # 1h 46m 56s

1048577 1048577
muling...
-> 1016470 1016471

1016470 1016471
muling...
-> 984363 984364

984363 984364
muling...
-> 952256 952257

952256 952257
muling...
-> 920149 920150

920149 920150
muling...
-> 888042 888043

888042 888043
muling...
-> 855935 855936

855935 855936
muling...
-> 823828 823829

823828 823829
muling...
-> 791721 791722

791721 791722
muling...
-> 759614 759615

759614 759615
muling...
-> 727507 727508

727507 727508
muling...
-> 695400 695401

695400 695401
muling...
-> 663293 663294

663293 663294
muling...
-> 631186 631187

631186 631187
muling...
-> 599079 599080

599079 599080
muling...
-> 566972 566973

566972 566973
muling...
-> 534865 534866

534865 534866
muling...
-> 502758 502759

502758 502759
muling...
-> 470651 470652

470651 470652
muling...
-> 438544 438545

438544 438545
muling...
-> 406437 406438

406437 406438
muling...
-> 374330 374331

374330 374331
muling...
-> 342223 342224

342223 342224
muling...
-> 310116 310117

310116 310117
muling...
-> 2

KeyboardInterrupt: 

In [ ]:
SOL = xgcd(a, b)[-1]
Bin(N-SOL.monic().subs(x=0)).bytes